# Dependencies

In [1]:
import numpy as np
import gymnasium as gym
import random 
import imageio
import os
import tqdm

import pickle
from tqdm.notebook import tqdm

# Q-table Class

In [2]:
class QTable:
    def __init__(self, state_space_size, action_space_size, eps_start = 1.0, eps_end = 0.0, decay_rate = 0.1, gamma=0.999, alpha=0.01):
        self.state_space_sz = state_space_size
        self.act_space_sz = action_space_size
        self.eps_start = eps_start
        self.eps_end= eps_end
        self.decay_rate = decay_rate
        self.gamma=gamma
        self.alpha = alpha
        self.reset()

    def reset(self):
        self.Qtable = np.zeros((self.state_space_sz, self.act_space_sz))

    def act_greedy_policy(self, curr_state):
        greedy_action = self.Qtable[curr_state, :].argmax()
        return greedy_action
    
    def act_eps_greedy_policy(self, curr_state, episode):
        random_number = random.uniform(0, 1)

        eps = self.eps_end + (self.eps_start - self.eps_end)*np.exp(-self.decay_rate*episode)

        if random_number > eps: # exploitation
            action = self.act_greedy_policy(curr_state)
        else: # exploration
            random_action = random.randint(0, self.act_space_sz-1)
            action = random_action

        return action
    
    def update(self, obs, act, next_obs, reward):
        self.Qtable[obs, act] = self.Qtable[obs, act] + self.alpha * (reward + self.gamma*self.Qtable[next_obs].max() - self.Qtable[obs, act])

# Frozen Lake

In [14]:
# Frozen Lake param descriptions
env_id = "FrozenLake-v1"  # Name of the environment
is_splippery = False
map_name = "4x4"
map_descr = ["SFFF", "FHFH", "FFFH", "HFFG"]
render_mode = "rgb_array"

In [15]:
env = gym.make(env_id, map_name=map_name, desc=map_descr, is_slippery=is_splippery, render_mode=render_mode)

print("---------- OBS SPACE ----------\n")
print("Obs space size: ", env.observation_space)
print("Sampling obs space: ", env.observation_space.sample())
print("---------- ACT SPACE ----------\n")
print("Act space size: ", env.action_space)
print("Sampling obs space: ", env.action_space.sample())

---------- OBS SPACE ----------

Obs space size:  Discrete(16)
Sampling obs space:  0
---------- ACT SPACE ----------

Act space size:  Discrete(4)
Sampling obs space:  0


In [16]:
env = gym.make(env_id, map_name=map_name, desc=map_descr, is_slippery=is_splippery, render_mode=render_mode)

state_space_size = env.observation_space.n
action_space_size = env.action_space.n
print("State space size is: ", state_space_size)
print("Action space size is: ", action_space_size)

n_training_episodes = 10000
learning_rate = 0.7

max_steps = 99
gamma = 0.95
eval_seed = []

max_epsilon = 1.0  # Exploration probability at start
min_epsilon = 0.05  # Minimum exploration probability
decay_rate = 0.0005  # Exponential decay rate for exploration prob

q_table = QTable(state_space_size, action_space_size, max_epsilon, min_epsilon, decay_rate, gamma, learning_rate)

# Train
for episode_num in range(0, n_training_episodes):
    obs, info = env.reset()

    for step_num in range(0, max_steps):
    
        act = q_table.act_eps_greedy_policy(obs, episode_num)
        next_obs, reward, terminated, truncated, info = env.step(action=act)
        q_table.update(obs, act, next_obs, reward)

        if truncated or terminated:
            break

        obs = next_obs
    
    print(f"Episode {episode_num} / {n_training_episodes} finished!")

env.close()

State space size is:  16
Action space size is:  4
Episode 0 / 10000 finished!
Episode 1 / 10000 finished!
Episode 2 / 10000 finished!
Episode 3 / 10000 finished!
Episode 4 / 10000 finished!
Episode 5 / 10000 finished!
Episode 6 / 10000 finished!
Episode 7 / 10000 finished!
Episode 8 / 10000 finished!
Episode 9 / 10000 finished!
Episode 10 / 10000 finished!
Episode 11 / 10000 finished!
Episode 12 / 10000 finished!
Episode 13 / 10000 finished!
Episode 14 / 10000 finished!
Episode 15 / 10000 finished!
Episode 16 / 10000 finished!
Episode 17 / 10000 finished!
Episode 18 / 10000 finished!
Episode 19 / 10000 finished!
Episode 20 / 10000 finished!
Episode 21 / 10000 finished!
Episode 22 / 10000 finished!
Episode 23 / 10000 finished!
Episode 24 / 10000 finished!
Episode 25 / 10000 finished!
Episode 26 / 10000 finished!
Episode 27 / 10000 finished!
Episode 28 / 10000 finished!
Episode 29 / 10000 finished!
Episode 30 / 10000 finished!
Episode 31 / 10000 finished!
Episode 32 / 10000 finished!
Epi

# Evaluation

In [38]:
# Evaluate the q_table
def evaluate_agent(env: gym.Env, q_table: QTable, max_steps = 99, n_eval_episodes = 100, seed = None):
    episode_rewards = []

    for episode in range(n_eval_episodes):

        if seed:
            obs, info = env.reset(seed=seed)
        else:
            obs, info = env.reset()

        episode_reward = 0

        for step in range(0, max_steps):
            
            act = q_table.act_greedy_policy(obs)
            obs, reward, terminated, truncated, _ = env.step(act)
            episode_reward = episode_reward + reward
            env.render()

            if terminated or truncated:
                break
        
        episode_rewards.append(episode_reward)

    mean_reward = np.mean(episode_rewards)
    std_reward = np.std(episode_rewards)

    return mean_reward, std_reward

env_eval = gym.make(env_id, map_name=map_name, desc=map_descr, is_slippery=is_splippery, render_mode=render_mode)
mean_reward, std_reward = evaluate_agent(env_eval, q_table)
print(f"Mean_reward={mean_reward:.2f} +/- {std_reward:.2f}")

Mean_reward=1.00 +/- 0.00


# Upload model to the Hub

In [39]:
from huggingface_hub import HfApi, snapshot_download
from huggingface_hub.repocard import metadata_eval_result, metadata_save

from pathlib import Path
import datetime
import json

In [40]:
def record_video(env: gym.Env, Qtable: QTable, seed, out_dir, fps=1):
    """
    Generate a replay video of the agent
    :param env
    :param Qtable: Qtable of our agent
    :param out_dir
    :param fps: how many frame per seconds (with taxi-v3 and frozenlake-v1 we use 1)
    """
    images = []
    terminated = False
    truncated = False

    state, info = env.reset(seed=seed)
    img = env.render()
    images.append(img)

    while not terminated or truncated:
        action = Qtable.act_greedy_policy(state)

        state, reward, terminated, truncated, info = env.step(action)
        img = env.render()
        images.append(img)

    imageio.mimsave(out_dir, [np.array(img) for i, img in enumerate(images)], fps=fps)

In [ ]:
def push_to_hub(repo_id, model, env: gym.Env, video_fps=1, local_repo_path="hub"):
    """
    Evaluate, Generate a video and Upload a model to Hugging Face Hub.
    This method does the complete pipeline:
    - It evaluates the model
    - It generates the model card
    - It generates a replay video of the agent
    - It pushes everything to the Hub

    :param repo_id: repo_id: id of the model repository from the Hugging Face Hub
    :param env
    :param video_fps: how many frame per seconds to record our video replay
    (with taxi-v3 and frozenlake-v1 we use 1)
    :param local_repo_path: where the local repository is
    """
    _, repo_name = repo_id.split("/")

    eval_env = env
    env_id = model["env_id"]
    api = HfApi()

    # Step 1: Create the repo
    repo_url = api.create_repo(repo_id = repo_id, exist_ok=True)

    # Step2: Download the files
    repo_local_path = Path(snapshot_download(repo_id=repo_id))

    # Step3: Save the model
    if env.spec.kwargs.get("map_name"):
        model["map_name"] = env.spec.kwargs.get("map_name")
        if env.spec.kwargs.get("is_splippery", "") == False:
            model["splippery"] = False

    # Pickle the model
    with open((repo_local_path) / "q-learning.pkl", "wb") as f:
        pickle.dump(model, f)

    # Step 4: Evaluate the model and build JSON with evaluation metrics
    mean_reward, std_reward = evaluate_agent(env, model["q_table"], model["max_steps"], model["n_eval_episodes"], model["seed"])

    evaluate_data = {
        "env_id": model["env_id"],
        "mean_reward": mean_reward,
        "n_eval_episodes": model["n_eval_episodes"],
        "eval_datetime": datetime.datetime.now().isoformat()
    }

    # Write a JSON file called "results.json" that will contain the
    # evaluation results
    with open(repo_local_path / "result.json", "w") as outfile:
        json.dump(evaluate_data, outfile)

    # Create the model card
    env_name = model["env_id"]
    if env.spec.kwargs.get("map_name"):
        env_name += "-" + env.spec.kwargs.get("map_name")

    if env.spec.kwargs.get("is_slippery", "") == False:
        env_name += "-" + "no_slippery"

    metadata = {}
    metadata["tags"] = [env_name, "q-learning", "reinforcement-learning", "custom-implememtation"] 

    eval = metadata_eval_result(
        model_pretty_name=repo_name,
        task_pretty_name="feinforcement-learning",
        task_id="reinforcment-learning",
        metrics_pretty_name="mean_reward",
        metrics_id="mean_reward",
        metrics_value=f"{mean_reward:.2f} +/- {std_reward:.2f}",
        dataset_pretty_name=env_name,
        dataset_id=env_name
    )

    # Merges both dictionaries
    metadata = {**metadata, **eval}

    model_card = f"""
    # **Q-Learning** Agent playing1 **{env_id}**
    This is a trained model of a **Q-Learning** agent playing **{env_id}** .

    ## Usage

    model = load_from_hub(repo_id="{repo_id}", filename="q-learning.pkl")

    # Don't forget to check if you need to add additional attributes (is_slippery=False etc)
    env = gym.make(model["env_id"])
    """

    readme_path = repo_local_path / "README.md"
    readme = ""
    print(readme_path.exists())
    if readme_path.exists():
        with readme_path.open("r", encoding="utf8") as f:
            readme = f.read()
    else:
        readme = model_card

    with readme_path.open("w", encoding="utf-8") as f:
        f.write(readme)

    # Save our metrics to Readme metadata
    metadata_save(readme_path, metadata)

    # Step 6: Record a video
    video_path = repo_local_path / "replay.mp4"
    record_video(env, q_table, model["seed"], video_path, video_fps)

    # Step 7. Push everything to the Hub
    api.upload_folder(
        repo_id=repo_id,
        folder_path=repo_local_path,
        path_in_repo=".",
    )

    print("Your model is pushed to the Hub. You can view your model here: ", repo_url)


In [44]:
from huggingface_hub import notebook_login

# Login
n_eval_episodes = 100

model = {
    "env_id": env_id,
    "max_steps": max_steps,
    "n_training_episodes": n_training_episodes,
    "n_eval_episodes": n_eval_episodes,
    "eval_seed": eval_seed,
    "learning_rate": learning_rate,
    "gamma": gamma,
    "max_epsilon": max_epsilon,
    "min_epsilon": min_epsilon,
    "decay_rate": decay_rate,
    "q_table": q_table,
    "seed": random.randint(0, 500)
}

username = "Pucciland95"
repo_name = "q-FrozenLake-v1-4x4-noSlippery"
push_to_hub(repo_id=f"{username}/{repo_name}", model=model, env=env)

Fetching 1 files: 100%|██████████| 1/1 [00:00<00:00, 2566.89it/s]


TypeError: metadata_eval_result() got an unexpected keyword argument 'metric_id'